## Mounting S3 Bucket to DataBricks

### Imports required for entire Batch Processing

In [ ]:
import urllib
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, TimestampType, DateType

### Retrieving AWS Credentials

In [ ]:
dbutils.fs.ls("/FileStore/tables")

Out[17]: [FileInfo(path='dbfs:/FileStore/tables/0ed442ca38ad_user_table/', name='0ed442ca38ad_user_table/', size=0, modificationTime=1703717932050),
 FileInfo(path='dbfs:/FileStore/tables/authentication.csv/', name='authentication.csv/', size=0, modificationTime=1703717932050),
 FileInfo(path='dbfs:/FileStore/tables/authentication_credentials-1.csv', name='authentication_credentials-1.csv', size=97, modificationTime=1702911405000),
 FileInfo(path='dbfs:/FileStore/tables/authentication_credentials-2.csv', name='authentication_credentials-2.csv', size=97, modificationTime=1702911558000),
 FileInfo(path='dbfs:/FileStore/tables/authentication_credentials-3.csv', name='authentication_credentials-3.csv', size=87, modificationTime=1703678451000),
 FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv/', name='authentication_credentials.csv/', size=0, modificationTime=1703717932050),
 FileInfo(path='dbfs:/FileStore/tables/geo_dirty.csv/', name='geo_dirty.csv/', size=0, modificationTime=1703717932050),
 FileInfo(path='dbfs:/FileStore/tables/pin_dirty.csv/', name='pin_dirty.csv/', size=0, modificationTime=1703717932051),
 FileInfo(path='dbfs:/FileStore/tables/single_json_file.json', name='single_json_file.json', size=293, modificationTime=1701857903000),
 FileInfo(path='dbfs:/FileStore/tables/user_dirty.csv/', name='user_dirty.csv/', size=0, modificationTime=1703717932051)]

In [ ]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [ ]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

### Mount the S3 Bucket

In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a40ea42f8d1-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0a40ea42f8d1-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[23]: True

In [ ]:
display(dbutils.fs.ls("/mnt/user-0a40ea42f8d1-bucket/../.."))

path,name,size,modificationTime
dbfs:/FileStore/,FileStore/,0,1703730274817
dbfs:/Volume/,Volume/,0,0
dbfs:/Volumes/,Volumes/,0,0
dbfs:/checkpoint/,checkpoint/,0,1703730274817
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/delta/,delta/,0,1703730274817
dbfs:/df_pin.csv/,df_pin.csv/,0,1703730274817
dbfs:/df_pin.parquet/,df_pin.parquet/,0,1703730274817
dbfs:/geo_dirty.csv/,geo_dirty.csv/,0,1703730274817


### Read contents of S3 bucket into dataframes and display them

In [ ]:

# File location and type
file_type = "json"
pin_file_path = "/mnt/user-0a40ea42f8d1-bucket/topics/0a40ea42f8d1.pin/partition=0/*.json"
geo_file_path = "/mnt/user-0a40ea42f8d1-bucket/topics/0a40ea42f8d1.geo/partition=0/*.json"
user_file_path = "/mnt/user-0a40ea42f8d1-bucket/topics/0a40ea42f8d1.user/partition=0/*.json"

# Ask Spark to infer the schema
infer_schema = "true"

# Read in JSONs from mounted S3 bucket
dirty_pin_df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(pin_file_path)

dirty_geo_df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(geo_file_path)

dirty_user_df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(user_file_path)

# Display Spark dataframe to check its content
display(dirty_pin_df)
display(dirty_geo_df)
display(dirty_user_df)

### Unmount S3 Bucket

In [ ]:
# If and when needed
dbutils.fs.unmount("/mnt/user-0a40ea42f8d1-bucket")

/mnt/user-0a40ea42f8d1-bucket has been unmounted.
Out[20]: True